In [50]:
import math
import numpy as np
import seaborn as sns
from time import time
import pandas as pd
import pymcmcstat
import matplotlib.pyplot as plt

In [114]:
def model_magnitud(theta, data):
    mag_v_sun = -26.72

    phase_angle = data['solar_phase_angle']
    range_sat = data['range_sat']
    cross_section = theta['cross_section']
    albedo = theta['albedo'] 
    mix_coef = theta['mix_coef']

   # Change degrees to radian
    degtorad = lambda x : x*np.pi/180.

   # Definition coefficients of equation
    coe_1 = 2/(3*np.pi)
    coe_2 = 1/(4*np.pi)

    phase_angle_rad = degtorad(phase_angle)	

   
    part1 = (coe_1 *((np.pi - phase_angle_rad)*np.cos(phase_angle_rad) - np.sin(phase_angle_rad))) - coe_2
   
    mag = mag_v_sun - 2.5 * np.log10(cross_section * albedo * (mix_coef * part1) + coe_2) + 5*np.log10(range_sat)
    return mag

In [72]:
 def residual_calc(theta, data_x, data_y):
    residual = data_y - model_magnitud(theta, data_x)
    return residual

### Read Data and create new dataframe

In [14]:
# path = '/home/kero/Documents/PhD/Re-analyse/result_analyse.csv'
path = '/home/kero/Documents/PhD/re-analyse/test.csv'
data_sat = pd.read_csv(path)

data_in = pd.DataFrame()
# data_in['obs_phase_angle'] = data_sat['obs_phase_angle']
# data_in['range_sat'] = data_sat['range_sat']
# data_in['mag_observation'] = data_sat['mag_observation']
data_in['solar_phase_angle'] = data_sat['SunPhaseAng']
data_in['range_sat'] = data_sat['ObsTrailLength']
data_in['mag_observation'] = data_sat['ObsMag']
data_in['e_mag'] = data_sat['e_ObsMag']
n = data_in.shape[0]

In [70]:
5*np.log10(data_in['range_sat'])

0    16.110501
1    16.152526
2    15.504687
3    13.873254
4    16.741181
5    16.652843
6    16.517102
7    13.246052
8    15.960430
Name: range_sat, dtype: float64

In [16]:
mean = 0.8
std_dev = 0.35
num_reps = n
num_simulations = 1000

cross_section_random = np.random.normal(mean, std_dev, num_reps).round(2)

In [21]:
mean = 0.55
std_dev = 0.2
num_reps = n
num_simulations = 1000

albedo_random = np.random.normal(mean, std_dev, num_reps).round(2)

array([0.31, 0.62, 0.47, 0.53, 0.74, 0.8 , 0.45, 0.57, 0.42])

In [18]:
mean = 0.75
std_dev = 0.12
num_reps = n
num_simulations = 1000

mix_coef_random = np.random.normal(mean, std_dev, num_reps).round(2)

In [118]:
def sum_squares(theta, data):

    rest = (data['mag_observation'] + data['e_mag']) + model_magnitud(theta, data)
    residual = (rest ** 2).sum(axis=0)/(rest.shape[0] - theta.shape[1])
    print('type residual',type(residual))
    estimator = [residual, rest]
    
    return estimator

In [121]:
for i in range(5):
    # Choose random inputs for the sales targets and percent to target
    mix_coef = np.random.choice(mix_coef_random)
    albedo = np.random.choice(albedo_random)
    cross_section = np.random.choice(cross_section_random)
    # print(mix_coef, albedo, cross_section)


    df = pd.DataFrame(index=range(num_reps), data={'mix_coef': mix_coef,
                                                    'albedo': albedo,
                                                    'cross_section' : cross_section})
    # print('Run model_magnitud')
    # print('===============================')
    model_value = model_magnitud(df, data_in)
    # # print(model_value)
    # print('End model_magnitud')
    # print('===============================')
    
    # print('Run sum_squares')
    # print('===============================')
    
    estimador = sum_squares(df, data_in)
    
    # print('End model_magnitud')
    # print('===============================')


type residual <class 'numpy.float64'>
type residual <class 'numpy.float64'>
type residual <class 'numpy.float64'>
type residual <class 'numpy.float64'>
type residual <class 'numpy.float64'>


/home/kero/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/kero/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/kero/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/home/kero/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [122]:
estimador

0   -0.488228
1    0.722993
2    1.489675
3   -0.104981
4    0.352782
5    3.450996
6         NaN
7         NaN
8    0.027127
dtype: float64